In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812366 sha256=8da1a57e0e7adf4dcddd6d9878f3141422cd4ac87af3125eeffbf2ea4098721e
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [5]:
import pyspark

# Creating a Spark Session

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab4").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 02:46:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Loading the data

In [8]:
path = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv"
df = spark.read.option("header", "True").csv(path, inferSchema = True)
df

DataFrame[event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string]

In [9]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [10]:
df.show()

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.s

# Question 1

In [11]:
df_cart_ratio = df.select(['category_id', 'event_type'])
df_cart_ratio.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- event_type: string (nullable = true)



In [12]:
df_cart_ratio.show()

+-------------------+----------+
|        category_id|event_type|
+-------------------+----------+
|2053013555631882655|      view|
|2053013566100866035|      view|
|2053013553853497655|      view|
|2053013563810775923|      view|
|2053013555631882655|      view|
|2053013558920217191|      view|
|2053013558920217191|      view|
|2053013558190408249|      view|
|2053013553559896355|      view|
|2053013555631882655|      view|
|2053013559792632471|      view|
|2053013563911439225|      view|
|2053013559901684381|      view|
|2060981320581906480|      view|
|2053013563944993659|      view|
|2053013562183385881|      view|
|2053013563391345499|      view|
|2053013553375346967|      view|
|2053013555631882655|      view|
|2053013558920217191|      view|
+-------------------+----------+
only showing top 20 rows



### There are 4 types of event_types : 
#### 1. view
#### 2. cart
#### 3. remove_from_cart
#### 4. purchase

In [20]:
df_view = df_cart_ratio.filter(df_cart_ratio["event_type"] == "view").groupBy('category_id').agg({'event_type' : 'count'})
df_view.show()

+-------------------+-----------------+
|        category_id|count(event_type)|
+-------------------+-----------------+
|2053013563944993659|           156018|
|2053013566209917945|           156130|
|2053013561554240247|            71114|
|2145727399348666855|             3828|
|2060237588744111062|             4022|
|2098563460336976001|             1574|
|2053013564968403895|            30807|
|2095518906859913319|            22797|
|2134905039313961849|            11808|
|2053013552737812719|             8068|
|2136389353511846005|             1114|
|2053013561747178243|             5808|
|2138568472580325973|             2959|
|2145727308156108933|              290|
|2173216765583032544|            19821|
|2053013555380224399|            48616|
|2053013563139687243|            69084|
|2090971680481476652|             1590|
|2053013557737423389|              493|
|2053013555631882655|         14832387|
+-------------------+-----------------+
only showing top 20 rows



## Seperate dataframes for view event_type and purchase event_type
### Grouped data on the basis of category id

In [22]:
df_view = df_view.withColumnRenamed("count(event_type)","view_count")
df_view.show()

+-------------------+----------+
|        category_id|view_count|
+-------------------+----------+
|2053013563944993659|    156018|
|2053013566209917945|    156130|
|2053013561554240247|     71114|
|2145727399348666855|      3828|
|2060237588744111062|      4022|
|2098563460336976001|      1574|
|2053013564968403895|     30807|
|2095518906859913319|     22797|
|2134905039313961849|     11808|
|2053013552737812719|      8068|
|2136389353511846005|      1114|
|2053013561747178243|      5808|
|2138568472580325973|      2959|
|2145727308156108933|       290|
|2173216765583032544|     19821|
|2053013555380224399|     48616|
|2053013563139687243|     69084|
|2090971680481476652|      1590|
|2053013557737423389|       493|
|2053013555631882655|  14832387|
+-------------------+----------+
only showing top 20 rows



In [24]:
df_view.count()

684

In [21]:
df_cart = df_cart_ratio.filter(df_cart_ratio["event_type"] == "cart").groupBy("category_id").agg({"event_type" : "count"})
df_cart.show()

+-------------------+-----------------+
|        category_id|count(event_type)|
+-------------------+-----------------+
|2053013563944993659|             3896|
|2053013552737812719|              503|
|2095518906859913319|              554|
|2053013563139687243|             2381|
|2053013555631882655|          1159966|
|2053013556277805513|             3547|
|2053013566142809077|             4325|
|2053013564003713919|            13149|
|2053013556344914381|             2463|
|2053013560346280633|             8491|
|2053013555355058573|             7521|
|2053013554692358509|              534|
|2053013556110033341|             1406|
|2053013561277416167|             8993|
|2053013558525952589|             3873|
|2053013553224352013|             1710|
|2053013554415534427|           106127|
|2053013554449088861|              546|
|2053013562082722579|             1036|
|2053013561092866779|             6847|
+-------------------+-----------------+
only showing top 20 rows



In [23]:
df_cart = df_cart.withColumnRenamed("count(event_type)","cart_count")
df_cart.show()

+-------------------+----------+
|        category_id|cart_count|
+-------------------+----------+
|2053013563944993659|      3896|
|2053013552737812719|       503|
|2095518906859913319|       554|
|2053013563139687243|      2381|
|2053013555631882655|   1159966|
|2053013556277805513|      3547|
|2053013566142809077|      4325|
|2053013564003713919|     13149|
|2053013556344914381|      2463|
|2053013560346280633|      8491|
|2053013555355058573|      7521|
|2053013554692358509|       534|
|2053013556110033341|      1406|
|2053013561277416167|      8993|
|2053013558525952589|      3873|
|2053013553224352013|      1710|
|2053013554415534427|    106127|
|2053013554449088861|       546|
|2053013562082722579|      1036|
|2053013561092866779|      6847|
+-------------------+----------+
only showing top 20 rows



In [25]:
df_cart.count()

647

In [29]:
df_cart_view = df_cart.join(df_view, df_cart.category_id == df_view.category_id, "inner")
df_cart_view.show()

+-------------------+----------+-------------------+----------+
|        category_id|cart_count|        category_id|view_count|
+-------------------+----------+-------------------+----------+
|2053013563944993659|      3896|2053013563944993659|    156018|
|2053013566209917945|      2380|2053013566209917945|    156130|
|2053013561554240247|      1771|2053013561554240247|     71114|
|2145727399348666855|       188|2145727399348666855|      3828|
|2060237588744111062|        14|2060237588744111062|      4022|
|2098563460336976001|        36|2098563460336976001|      1574|
|2053013564968403895|       451|2053013564968403895|     30807|
|2095518906859913319|       554|2095518906859913319|     22797|
|2134905039313961849|       138|2134905039313961849|     11808|
|2053013552737812719|       503|2053013552737812719|      8068|
|2136389353511846005|         9|2136389353511846005|      1114|
|2053013561747178243|       157|2053013561747178243|      5808|
|2138568472580325973|        15|21385684

In [30]:
df_cart_view.count()

647

### Ratio of cart_count/view_count will be taken, because we want the rate at which the product which was viewed was also converted to cart

In [31]:
df_ratio = df_cart_view.withColumn("conversion_ratio", df_cart_view["cart_count"]/df_cart_view["view_count"])
df_ratio.show()

+-------------------+----------+-------------------+----------+--------------------+
|        category_id|cart_count|        category_id|view_count|    conversion_ratio|
+-------------------+----------+-------------------+----------+--------------------+
|2053013563944993659|      3896|2053013563944993659|    156018|0.024971477650014742|
|2053013566209917945|      2380|2053013566209917945|    156130|0.015243707167104336|
|2053013561554240247|      1771|2053013561554240247|     71114|0.024903675788171105|
|2145727399348666855|       188|2145727399348666855|      3828| 0.04911180773249739|
|2060237588744111062|        14|2060237588744111062|      4022|  0.0034808552958727|
|2098563460336976001|        36|2098563460336976001|      1574|0.022871664548919948|
|2053013564968403895|       451|2053013564968403895|     30807| 0.01463952997695329|
|2095518906859913319|       554|2095518906859913319|     22797| 0.02430144317234724|
|2134905039313961849|       138|2134905039313961849|     11808|0.

## Finding the maximum conversion ratio from the column values

In [36]:
df_max_cat_id = df_ratio.select(f.max(df_ratio["conversion_ratio"]))
df_max_cat_id.show()

+---------------------+
|max(conversion_ratio)|
+---------------------+
|  0.10133399618858231|
+---------------------+



In [39]:
max_conversion_rate = df_max_cat_id.collect()
max_conversion_rate

[Row(max(conversion_ratio)=0.10133399618858231)]

In [42]:
max_conversion_rate[0][0]

0.10133399618858231

### Finding the category with maximum conversion rate


In [46]:
df_cat = df_ratio.filter(df_ratio["conversion_ratio"] == max_conversion_rate[0][0])
df_cat.show()

+-------------------+----------+-------------------+----------+-------------------+
|        category_id|cart_count|        category_id|view_count|   conversion_ratio|
+-------------------+----------+-------------------+----------+-------------------+
|2053013552662315243|      3669|2053013552662315243|     36207|0.10133399618858231|
+-------------------+----------+-------------------+----------+-------------------+



In [49]:
df_cat_coll = df_cat.collect()
df_cat_coll

[Row(category_id=2053013552662315243, cart_count=3669, category_id=2053013552662315243, view_count=36207, conversion_ratio=0.10133399618858231)]

In [50]:
df_cat_coll[0][0] # category with maximum conversion ratio

2053013552662315243

# Question 2

In [14]:
df_view_ratio = df.select(['event_type', 'brand', 'event_time'])
df_view_ratio.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- event_time: timestamp (nullable = true)



In [15]:
df_view_ratio.show()

+----------+--------+-------------------+
|event_type|   brand|         event_time|
+----------+--------+-------------------+
|      view|  xiaomi|2019-11-01 00:00:00|
|      view|  janome|2019-11-01 00:00:00|
|      view|   creed|2019-11-01 00:00:01|
|      view|      lg|2019-11-01 00:00:01|
|      view|  xiaomi|2019-11-01 00:00:01|
|      view|      hp|2019-11-01 00:00:01|
|      view|      hp|2019-11-01 00:00:01|
|      view| rondell|2019-11-01 00:00:02|
|      view|michelin|2019-11-01 00:00:02|
|      view|   apple|2019-11-01 00:00:02|
|      view|    NULL|2019-11-01 00:00:03|
|      view|    NULL|2019-11-01 00:00:03|
|      view|    NULL|2019-11-01 00:00:03|
|      view|    NULL|2019-11-01 00:00:04|
|      view| samsung|2019-11-01 00:00:05|
|      view|    NULL|2019-11-01 00:00:06|
|      view|    NULL|2019-11-01 00:00:07|
|      view|  xiaomi|2019-11-01 00:00:07|
|      view|  huawei|2019-11-01 00:00:07|
|      view|      hp|2019-11-01 00:00:08|
+----------+--------+-------------

In [32]:
import pyspark.sql.functions as f
df_view_ratio = df_view_ratio.withColumn("time", f.date_format("event_time", "HH::mm::ss"))
df_view_ratio.show()

+----------+--------+-------------------+----------+
|event_type|   brand|         event_time|      time|
+----------+--------+-------------------+----------+
|      view|  xiaomi|2019-11-01 00:00:00|00::00::00|
|      view|  janome|2019-11-01 00:00:00|00::00::00|
|      view|   creed|2019-11-01 00:00:01|00::00::01|
|      view|      lg|2019-11-01 00:00:01|00::00::01|
|      view|  xiaomi|2019-11-01 00:00:01|00::00::01|
|      view|      hp|2019-11-01 00:00:01|00::00::01|
|      view|      hp|2019-11-01 00:00:01|00::00::01|
|      view| rondell|2019-11-01 00:00:02|00::00::02|
|      view|michelin|2019-11-01 00:00:02|00::00::02|
|      view|   apple|2019-11-01 00:00:02|00::00::02|
|      view|    NULL|2019-11-01 00:00:03|00::00::03|
|      view|    NULL|2019-11-01 00:00:03|00::00::03|
|      view|    NULL|2019-11-01 00:00:03|00::00::03|
|      view|    NULL|2019-11-01 00:00:04|00::00::04|
|      view| samsung|2019-11-01 00:00:05|00::00::05|
|      view|    NULL|2019-11-01 00:00:06|00::0

In [37]:
df_view_ratio.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- time: string (nullable = true)



In [47]:
df_time = df_view_ratio.withColumn("hour", f.hour("event_time"))
df_time.show()

+----------+--------+-------------------+----------+----+
|event_type|   brand|         event_time|      time|hour|
+----------+--------+-------------------+----------+----+
|      view|  xiaomi|2019-11-01 00:00:00|00::00::00|   0|
|      view|  janome|2019-11-01 00:00:00|00::00::00|   0|
|      view|   creed|2019-11-01 00:00:01|00::00::01|   0|
|      view|      lg|2019-11-01 00:00:01|00::00::01|   0|
|      view|  xiaomi|2019-11-01 00:00:01|00::00::01|   0|
|      view|      hp|2019-11-01 00:00:01|00::00::01|   0|
|      view|      hp|2019-11-01 00:00:01|00::00::01|   0|
|      view| rondell|2019-11-01 00:00:02|00::00::02|   0|
|      view|michelin|2019-11-01 00:00:02|00::00::02|   0|
|      view|   apple|2019-11-01 00:00:02|00::00::02|   0|
|      view|    NULL|2019-11-01 00:00:03|00::00::03|   0|
|      view|    NULL|2019-11-01 00:00:03|00::00::03|   0|
|      view|    NULL|2019-11-01 00:00:03|00::00::03|   0|
|      view|    NULL|2019-11-01 00:00:04|00::00::04|   0|
|      view| s

In [51]:
df_time = df_time.withColumn("minute", f.minute("event_time"))
df_time = df_time.withColumn("seconds", f.second("event_time"))

In [52]:
df_time.show()

+----------+--------+-------------------+----------+----+------+-------+
|event_type|   brand|         event_time|      time|hour|minute|seconds|
+----------+--------+-------------------+----------+----+------+-------+
|      view|  xiaomi|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|  janome|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|   creed|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|      lg|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|  xiaomi|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|      hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|      hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view| rondell|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|michelin|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|   apple|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|    NULL|2019-11-01 00:00:03|00::00::03

In [53]:
df_time.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- time: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- seconds: integer (nullable = true)



In [57]:
df_midnight_6 = df_time.filter(((df_time["hour"] >= 0 ) & (df_time["hour"] <= 5)) | ((df_time["hour"] == 6) & (df_time["minute"] == 0) & (df_time["seconds"] == 0)))
df_midnight_6.show(100)

+----------+----------+-------------------+----------+----+------+-------+
|event_type|     brand|         event_time|      time|hour|minute|seconds|
+----------+----------+-------------------+----------+----+------+-------+
|      view|    xiaomi|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|    janome|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|     creed|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|        lg|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|    xiaomi|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|        hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|        hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|   rondell|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|  michelin|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|     apple|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|      NULL|20

In [60]:
df_midnight_6 = df_midnight_6.na.drop(subset = ["brand"])
df_midnight_6.show()

+----------+---------+-------------------+----------+----+------+-------+
|event_type|    brand|         event_time|      time|hour|minute|seconds|
+----------+---------+-------------------+----------+----+------+-------+
|      view|   xiaomi|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|   janome|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|    creed|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|       lg|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|   xiaomi|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|       hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|       hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|  rondell|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view| michelin|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|    apple|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|  samsung|2019-11-01 00:00

In [61]:
df_view_brand = df_midnight_6.filter(df["event_type"] == "view")
df_purchase_brand = df_midnight_6.filter(df["event_type"] == "purchase")

In [63]:
df_view_brand.show()

+----------+---------+-------------------+----------+----+------+-------+
|event_type|    brand|         event_time|      time|hour|minute|seconds|
+----------+---------+-------------------+----------+----+------+-------+
|      view|   xiaomi|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|   janome|2019-11-01 00:00:00|00::00::00|   0|     0|      0|
|      view|    creed|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|       lg|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|   xiaomi|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|       hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|       hp|2019-11-01 00:00:01|00::00::01|   0|     0|      1|
|      view|  rondell|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view| michelin|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|    apple|2019-11-01 00:00:02|00::00::02|   0|     0|      2|
|      view|  samsung|2019-11-01 00:00

In [64]:
df_purchase_brand.show()

+----------+----------+-------------------+----------+----+------+-------+
|event_type|     brand|         event_time|      time|hour|minute|seconds|
+----------+----------+-------------------+----------+----+------+-------+
|  purchase|    xiaomi|2019-11-01 00:01:04|00::01::04|   0|     1|      4|
|  purchase|   samsung|2019-11-01 00:04:51|00::04::51|   0|     4|     51|
|  purchase|   lucente|2019-11-01 00:05:34|00::05::34|   0|     5|     34|
|  purchase|   samsung|2019-11-01 00:06:33|00::06::33|   0|     6|     33|
|  purchase| nakamichi|2019-11-01 00:06:34|00::06::34|   0|     6|     34|
|  purchase|  magnetta|2019-11-01 00:07:38|00::07::38|   0|     7|     38|
|  purchase|    xiaomi|2019-11-01 00:09:30|00::09::30|   0|     9|     30|
|  purchase|   samsung|2019-11-01 00:10:12|00::10::12|   0|    10|     12|
|  purchase|  cortland|2019-11-01 00:10:38|00::10::38|   0|    10|     38|
|  purchase|     apple|2019-11-01 00:10:47|00::10::47|   0|    10|     47|
|  purchase|    maxxis|20

In [66]:
df_view_brand = df_view_brand.groupBy("brand").agg({"brand":"count"})
df_view_brand.show()

+-------------+------------+
|        brand|count(brand)|
+-------------+------------+
|     yokohama|       25666|
|      edifier|         570|
|        welss|         774|
|      tuffoni|         511|
|         tega|         366|
|      serebro|        1240|
|        crest|          46|
|         tmnt|          73|
|       alutec|         410|
|        sonel|        1901|
|     nocnezna|          39|
|     nutricia|         380|
|      bombbar|         142|
|trianglegroup|          28|
|      keenway|         419|
|        sigma|         448|
|        goo.n|         159|
|      fitwell|         102|
| belaakalitva|          50|
|       marley|         188|
+-------------+------------+
only showing top 20 rows



In [68]:
df_view_brand = df_view_brand.withColumnRenamed("count(brand)", "view_brand_count")
df_view_brand.printSchema()

root
 |-- brand: string (nullable = true)
 |-- view_brand_count: long (nullable = false)



In [67]:
df_purchase_brand = df_purchase_brand.groupBy("brand").agg({"brand":"count"})
df_purchase_brand.show()

+-------------------+------------+
|              brand|count(brand)|
+-------------------+------------+
|           yokohama|         620|
|               tega|           3|
|           nutricia|          13|
|            ariston|         327|
|           decoroom|          28|
|            versace|          96|
|             rieker|         108|
|            philips|         716|
|             yamaha|          97|
|            greyder|          42|
|               pupa|          16|
|            dxracer|          11|
|          microsoft|          34|
|           cordiant|        3850|
|             jaguar|          52|
|              remax|          23|
|            matador|         523|
|geneticlabnutrition|           1|
|               bene|          14|
|              olmio|          17|
+-------------------+------------+
only showing top 20 rows



In [69]:
df_purchase_brand = df_purchase_brand.withColumnRenamed("count(brand)", "purchase_brand_count")
df_purchase_brand.printSchema()

root
 |-- brand: string (nullable = true)
 |-- purchase_brand_count: long (nullable = false)



### Inner join on the above 2 tables to get the ratio of purchase/view conversion rate from view to purchase for products

In [71]:
df_view_purchase = df_view_brand.join(df_purchase_brand, df_view_brand["brand"] == df_purchase_brand["brand"], "inner")
df_view_purchase.show()

+-------------------+----------------+-------------------+--------------------+
|              brand|view_brand_count|              brand|purchase_brand_count|
+-------------------+----------------+-------------------+--------------------+
|           yokohama|           25666|           yokohama|                 620|
|               tega|             366|               tega|                   3|
|           nutricia|             380|           nutricia|                  13|
|            ariston|           18578|            ariston|                 327|
|           decoroom|            2237|           decoroom|                  28|
|            versace|           10611|            versace|                  96|
|             rieker|           28952|             rieker|                 108|
|            philips|           76293|            philips|                 716|
|             yamaha|           11526|             yamaha|                  97|
|            greyder|           13020|  

In [72]:
df_ratio_view_purchase = df_view_purchase.withColumn("Conversion rate (purchase/view)", df_view_purchase["purchase_brand_count"]/df_view_purchase["view_brand_count"])
df_ratio_view_purchase.show()

+-------------------+----------------+-------------------+--------------------+-------------------------------+
|              brand|view_brand_count|              brand|purchase_brand_count|Conversion rate (purchase/view)|
+-------------------+----------------+-------------------+--------------------+-------------------------------+
|           yokohama|           25666|           yokohama|                 620|            0.02415647159666485|
|               tega|             366|               tega|                   3|            0.00819672131147541|
|           nutricia|             380|           nutricia|                  13|           0.034210526315789476|
|            ariston|           18578|            ariston|                 327|            0.01760146409731941|
|           decoroom|            2237|           decoroom|                  28|           0.012516763522574878|
|            versace|           10611|            versace|                  96|           0.009047215154

In [73]:
df_max_ratio = df_ratio_view_purchase.select(f.max(df_ratio_view_purchase["Conversion rate (purchase/view)"]))
df_max_ratio.show()

+------------------------------------+
|max(Conversion rate (purchase/view))|
+------------------------------------+
|                  0.3333333333333333|
+------------------------------------+



In [74]:
max_view_purchase = df_max_ratio.collect()[0][0]
max_view_purchase

0.3333333333333333

In [76]:
df_filtered = df_ratio_view_purchase.filter(df_ratio_view_purchase["Conversion rate (purchase/view)"] == max_view_purchase)
df_filtered.show()

+-----------+----------------+-----------+--------------------+-------------------------------+
|      brand|view_brand_count|      brand|purchase_brand_count|Conversion rate (purchase/view)|
+-----------+----------------+-----------+--------------------+-------------------------------+
|pixiebelles|               3|pixiebelles|                   1|             0.3333333333333333|
+-----------+----------------+-----------+--------------------+-------------------------------+



# Question 3

In [38]:
df_cat_comp = df.select(["brand", "category_id", "event_type", "event_time"])
df_cat_comp.show()

+--------+-------------------+----------+-------------------+
|   brand|        category_id|event_type|         event_time|
+--------+-------------------+----------+-------------------+
|  xiaomi|2053013555631882655|      view|2019-11-01 00:00:00|
|  janome|2053013566100866035|      view|2019-11-01 00:00:00|
|   creed|2053013553853497655|      view|2019-11-01 00:00:01|
|      lg|2053013563810775923|      view|2019-11-01 00:00:01|
|  xiaomi|2053013555631882655|      view|2019-11-01 00:00:01|
|      hp|2053013558920217191|      view|2019-11-01 00:00:01|
|      hp|2053013558920217191|      view|2019-11-01 00:00:01|
| rondell|2053013558190408249|      view|2019-11-01 00:00:02|
|michelin|2053013553559896355|      view|2019-11-01 00:00:02|
|   apple|2053013555631882655|      view|2019-11-01 00:00:02|
|    NULL|2053013559792632471|      view|2019-11-01 00:00:03|
|    NULL|2053013563911439225|      view|2019-11-01 00:00:03|
|    NULL|2053013559901684381|      view|2019-11-01 00:00:03|
|    NUL

In [81]:
df_cat_comp = df_cat_comp.na.drop(subset = ["brand"])

In [82]:
df_time = df_cat_comp.withColumn("hour", f.hour("event_time"))
df_time = df_time.withColumn("minute", f.minute("event_time"))
df_time = df_time.withColumn("seconds", f.second("event_time"))

In [83]:
df_time_midnight_6 = df_time.filter(((df_time["hour"] >= 0 ) & (df_time["hour"] <= 5)) | ((df_time["hour"] == 6) & (df_time["minute"] == 0) & (df_time["seconds"] == 0)))

In [85]:
brand_name = "pixiebelles"

In [86]:
df_comp = df_time_midnight_6.filter(df["brand"] == brand_name)
df_comp.show()

+-----------+-------------------+----------+-------------------+----+------+-------+
|      brand|        category_id|event_type|         event_time|hour|minute|seconds|
+-----------+-------------------+----------+-------------------+----+------+-------+
|pixiebelles|2053013553199186187|      view|2019-11-29 03:31:57|   3|    31|     57|
|pixiebelles|2053013553199186187|      cart|2019-11-29 03:42:45|   3|    42|     45|
|pixiebelles|2053013553199186187|      view|2019-11-29 03:45:28|   3|    45|     28|
|pixiebelles|2053013553199186187|      view|2019-11-29 03:47:04|   3|    47|      4|
|pixiebelles|2053013553199186187|      cart|2019-11-29 03:47:09|   3|    47|      9|
|pixiebelles|2053013553199186187|  purchase|2019-11-29 03:47:34|   3|    47|     34|
+-----------+-------------------+----------+-------------------+----+------+-------+



In [87]:
df_view_comp = df_comp.filter(df["event_type"] == "view")
df_purchase_comp = df_comp.filter(df["event_type"] == "purchase")

In [98]:
df_view = df_view_comp.groupBy("category_id").agg(f.count("category_id").alias("view_category_count"))

In [99]:
df_view.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- view_category_count: long (nullable = false)



In [123]:
df_purchase = df_purchase_comp.groupBy("category_id").agg(f.count("category_id").alias("purchase_category_count"))

In [124]:
df_purchase.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- purchase_category_count: long (nullable = false)



In [125]:
df_purchase = df_purchase.withColumnRenamed("category_id", "purchase_cat")
df_view = df_view.withColumnRenamed("category_id", "view_cat")

In [126]:
df_view.printSchema()

root
 |-- view_cat: long (nullable = true)
 |-- view_category_count: long (nullable = false)



In [127]:
df_purchase.printSchema()

root
 |-- purchase_cat: long (nullable = true)
 |-- purchase_category_count: long (nullable = false)



In [129]:
df_view_purchase_comp = df_purchase.join(df_view, df_view["view_cat"] == df_purchase["purchase_cat"], "inner")
df_view_purchase_comp.printSchema()

root
 |-- purchase_cat: long (nullable = true)
 |-- purchase_category_count: long (nullable = false)
 |-- view_cat: long (nullable = true)
 |-- view_category_count: long (nullable = false)



In [130]:
df_view_purchase_comp = df_view_purchase_comp.withColumn("ratio", df_view_purchase_comp["purchase_category_count"]/df_view_purchase_comp["view_category_count"])
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [ ]:
df_view_purchase = df_view_purchase_comp.select(f.max(df_view_purchase_comp["ratio"]))
max_ratio = df_view_purchase.collect()[0][0]
max_ratio

In [ ]:
df_view_purchase.printSchema()

In [ ]:
df_cat_max_ratio = df_view_purchase.filter(df_view_purchase_comp["max(ratio)"] == max_ratio)

In [ ]:
df_cat_max_ratio.printSchema()

In [ ]:
df_cat_max_ratio.select("view_cat").show()

# Question 4

In [113]:
df_purchase = df.filter(df["event_type"] == "purchase")

In [114]:
df_purchase = df_purchase.groupBy("category_id").agg({"price":"sum"})
df_max_price = df_purchase.select(f.max("sum(price)"))
max_price = df_max_price.collect()[0][0]

In [116]:
df_purchase.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- sum(price): double (nullable = true)



In [117]:
df_cat.printSchema()

root
 |-- category_id: long (nullable = true)
 |-- sum(price): double (nullable = true)



In [118]:
df_cat = df_purchase.filter(df_purchase["sum(price)"] == max_price)
df_cat.select("category_id").show()

+-------------------+
|        category_id|
+-------------------+
|2053013555631882655|
+-------------------+

